In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 定义超参数
num_epochs = 2
num_classes = 10
batch_size = 128
learning_rate = 0.001

# 加载数据集
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified


In [2]:
# 定义LeNet模型
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out


In [3]:
# 选择模型
model = LeNet(num_classes=num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
total_step = len(train_loader)
train_loss_list, train_acc_list, test_loss_list, test_acc_list = [], [], [], []
for epoch in range(num_epochs):
    train_loss, train_acc, test_loss, test_acc = 0, 0, 0, 0
    for i, data in enumerate(train_loader):

        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # 前向传播  (images, labels)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 计算训练的损失和精度
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_acc += (predicted == labels).sum().item()

    # 在测试集上评估模型
    with torch.no_grad():
        for images, labels in test_loader:
          # images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)

          # 计算测试的损失和精度
          test_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          test_acc += (predicted == labels).sum().item()

    train_loss_list.append(train_loss / len(train_loader))
    train_acc_list.append(train_acc / len(train_loader.dataset))
    test_loss_list.append(test_loss / len(test_loader))
    test_acc_list.append(test_acc / len(test_loader.dataset))

    # 打印训练过程中的数据
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.4f}'
          .format(epoch + 1, num_epochs, train_loss / len(train_loader), train_acc / len(train_loader.dataset),test_loss / len(test_loader), test_acc / len(test_loader.dataset)))

# 绘制损失率和准确率曲线
epoch_list = range(1, num_epochs + 1)
plt.plot(epoch_list, train_loss_list, 'b-', epoch_list, test_loss_list, 'r-')
plt.legend(['Training Loss', 'Testing Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

plt.plot(epoch_list, train_acc_list, 'b-', epoch_list, test_acc_list, 'r-')
plt.legend(['Training Accuracy', 'Testing Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

Epoch [1/2], Train Loss: 1.8818, Train Acc: 0.3195, Test Loss: 1.7086, Test Acc: 0.3926


KeyboardInterrupt: 